# Was sind die größten und komplexesten Methoden?

## 1. Verbindung zur Datenbank
Es wird eine Verbindung zur Neo4j-Datenbank aufgebaut.

In [ ]:
import py2neo

graph = py2neo.Graph(bolt=True, host='localhost', user='neo4j', password='neo4j')

## 2. Cypher-Abfrage
Es wird eine Abfrage an die Datenbank gestellt. Das Ergebnis wird in einem Dataframe (pandas) gespeichert.

In [ ]:
import pandas as pd

query ="MATCH (t:Type)-[:HAS_SOURCE]->(f), (t)-[:DECLARES]->(m:Method) RETURN DISTINCT t.fqn as type, m.signature as signature,  m.effectiveLineCount as loc, m.cyclomaticComplexity as complexity"
df = pd.DataFrame(graph.run(query).data())


## 3. Datenaufbereitung
Zur Kontrolle werden die ersten fünf Zeilen des Ergebnisses der Abfrage als Tabelle ausgegeben.

In [ ]:
df.head()

Im folgenden Codeabschnitt wird das Dataframe absteigend entsprechend der Anzahl der Quelltextzeilen (loc) mit der Methode ```sort_values``` sortiert und die ersten 10 Zeilen der Spalten ```loc```, ```signature``` und ```type``` im Dataframe ```loc_df``` abgebildet (https://www.data-science-architect.de/selektieren-von-daten-in-dataframes/).

In [ ]:
# Sortiere das Dataframe nach der Anzahl der Quelltextzeilen (loc).
df = df.sort_values('loc',ascending=False)
# Extrahiere die Spalten loc, signature und type und speichere sie im Dataframe loc_df.
loc_df = df[['loc','signature','type']][0:10]
loc_df.head(10)

Im folgenden Codeabschnitt wird das Dataframe absteigend entsprechend der Komplexität (complexity) mit der Methode ```sort_values``` sortiert und die ersten 10 Zeilen der Spalten ```complexity```, ```signature``` und ```type``` im Dataframe ```complexity_df``` abgebildet (https://www.data-science-architect.de/selektieren-von-daten-in-dataframes/).

In [ ]:
# Sortiere das Dataframe nach der zyklomatischen Komplexität (complexity).
df = df.sort_values('complexity',ascending=False)

# Extrahiere die Spalten complexity, signature und type und speichere sie im Dataframe complexity_df.
complexity_df = df[['complexity','signature','type']][0:10]
complexity_df.head(10)


## 4. Visualisierung
Die größten und komplexesten Methoden werden jeweils mit einem horizontalen Balkendiagramm visualisiert.

In [ ]:
from IPython.display import display, HTML

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

In [ ]:
# Erstelle Bar Chart für loc.
import pygal
bar_chart = pygal.HorizontalBar(show_legend=True, human_readable=True, fill=True, legend_at_bottom=True, legend_at_bottom_columns=2)
bar_chart.title = 'Größte Methoden'
for index, row in loc_df.iterrows():
    bar_chart.add(row['type'],[{"value": row['loc'], "label": row['signature']}])
display(HTML(base_html.format(rendered_chart=bar_chart.render(is_unicode=True))))

In [ ]:
# Erstelle Bar Chart für complexity.
import pygal 
bar_chart = pygal.HorizontalBar(show_legend=True, human_readable=True, fill=True, legend_at_bottom=True, legend_at_bottom_columns=2)
bar_chart.title = 'Komplexeste Methoden'
for index, row in complexity_df.iterrows():
    bar_chart.add(row['type'],[{"value": row['complexity'], "label": row['signature']}])
display(HTML(base_html.format(rendered_chart=bar_chart.render(is_unicode=True))))